In [1]:
#encoding=utf-8
import pandas as pd
import numpy as np
import re
import string

In [2]:
# path = '/home/kai/data/jiangning/Chatbot_1/Chatbot1.0/data/'
path = '../../../data/Installment/'
# data = pd.read_csv(path + 'mock_up_data.csv', encoding='utf8')
data = pd.read_csv(path + 'mock_up_data1.csv', encoding='utf8')

# data = data.drop(['id','text','备注'], axis = 1)
data = data.dropna()
data.head(10)

,label,split_text
0,0,那 时间 肯定 够了
1,1,估计 不行
2,2,诶 你 再 说一次
3,0,行
4,0,好的
5,0,这样 应该 没问题 了
6,0,那 我 会 按照 分期 按时 还款 的
7,0,我 会 按时 还款 的
8,0,我 会 按照 这个 准时 还款 的
9,0,没问题


# use goslate to get more data

In [3]:
from textblob import TextBlob
from textblob.translate import NotTranslated
import pandas as pd
import numpy as np
from multiprocessing import Pool
from itertools import repeat

In [4]:
def translate(comment, from_lang, to_lang):
        if hasattr(comment, "decode"):
            comment = comment.decode("utf-8")

        text = TextBlob(comment)
        try:
            text = text.translate(to=to_lang)
            text = text.translate(to=from_lang)
        except NotTranslated:
            pass

        return str(text)

def translate_csv(df,col,from_lang,to_lang,num_pol=10):
    """
        https://developers.google.com/translate/v2/using_rest#language-params
    """

    df = df.copy()

    if num_pol >= 1000:
        num_pol=1000
    comment_pool = df[col].values
    p = Pool(num_pol)
    new_col_name = col + '_' + to_lang
    df[new_col_name] = p.starmap(translate, zip(comment_pool, repeat(from_lang),repeat(to_lang)))
    df = df.drop([col], axis = 1)
    df = df.rename(index=str, columns={new_col_name:col})
    return df

In [5]:
col = 'split_text'
data_en = translate_csv(data,col,from_lang='zh',to_lang='en',num_pol=50)
print('finish 1st trans')
data_fr = translate_csv(data,col,from_lang='zh',to_lang='fr',num_pol=50)
print('finish 2nd trans')
data_th = translate_csv(data,col,from_lang='zh',to_lang='th',num_pol=50)
print('finish 3rd trans')
data_lo = translate_csv(data,col,from_lang='zh',to_lang='lo',num_pol=50)
print('finish 4th trans')

finish 1st trans
finish 2nd trans
finish 3rd trans
finish 4th trans


In [6]:
data = pd.concat([data,data_en,data_fr,data_th,data_lo], ignore_index=True)
data

,label,split_text
0,0,那 时间 肯定 够了
1,1,估计 不行
2,2,诶 你 再 说一次
3,0,行
4,0,好的
5,0,这样 应该 没问题 了
6,0,那 我 会 按照 分期 按时 还款 的
7,0,我 会 按时 还款 的
8,0,我 会 按照 这个 准时 还款 的
9,0,没问题


In [7]:
import jieba
jieba.load_userdict("../../WordCut/userdict.txt")
def cut_words(text):
    seg_list = jieba.cut(text, cut_all=False)
    return " ".join(seg_list)

data['split_text']=data['split_text'].apply(cut_words)
data

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.479 seconds.
Prefix dict has been built succesfully.


,label,split_text
0,0,那 时间 肯定 够 了
1,1,估计 不行
2,2,诶 你 再 说 一次
3,0,行
4,0,好 的
5,0,这样 应该 没 问题 了
6,0,那 我 会 按照 分期 按时 还款 的
7,0,我 会 按时 还款 的
8,0,我 会 按照 这个 准时 还款 的
9,0,没 问题


# data cleaning and save

In [8]:
def clean(text):
    text = re.sub(f'([{string.punctuation}“”¨«»®´·º ½¾¿¡§£₤‘’，])',' ', text)
    text = text.split(' ')
    text = ' '.join(text)
    return text


def clean_label(label):
    return int(label)
    
print('split_text cleaning')
# data['split_text'].head(10)
data['split_text'] = data['split_text'].apply(clean)
data['label'] = data['label'].apply(clean_label)

# shuffle data
data = data.sample(frac=1).reset_index(drop=True)

data.to_csv(path + 'cleaned_mock_up_data.csv', index = False, encoding = 'utf8')

split_text cleaning


In [9]:
data.label.values

array([2, 0, 2, ..., 1, 0, 0])